##Самостоятельная работа
##Требуется выяснить:

##Какое соотношение сторон экрана телефона самое популярное, плотность пикселей у экрана.
##Можно использовать только rdd.

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=136b09a2c489390d284ede4997091828b706000015642dabe20047b94cdc6b0f
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '4g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext


# Вспомогательная функция

def beautiful_print(rdd, num, title):
    print(title)
    for row in rdd.take(num):
        print('\t', row)

In [3]:
train = sc.textFile('/content/train.csv')
beautiful_print(train, 3, 'Train:')
train_first_row = train.first()
print(f"Train size: {train.count()} x {len(train_first_row.split(','))}")

Train:
	 battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
	 842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1
	 1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2
Train size: 2001 x 21


## Задание 1
###  Прочитайте test.csv

In [4]:
test = sc.textFile('/content/test.csv')
beautiful_print(test, 3, 'Test:')
test_first_row = test.first()
print(f"Test size: {test.count()} x {len(test_first_row.split(','))}")

Test:
	 id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
	 1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0
	 2,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0
Test size: 1001 x 21


###Преобразуем train и test

In [5]:
train = train\
    .filter(lambda row: row != train_first_row)\
    .map(lambda row: [float(el) for el in row.split(',')[:-1]])
    
train_header = train_first_row.split(',')
beautiful_print(train, 3, train_header)

['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g', 'touch_screen', 'wifi', 'price_range']
	 [842.0, 0.0, 2.2, 0.0, 1.0, 0.0, 7.0, 0.6, 188.0, 2.0, 2.0, 20.0, 756.0, 2549.0, 9.0, 7.0, 19.0, 0.0, 0.0, 1.0]
	 [1021.0, 1.0, 0.5, 1.0, 0.0, 1.0, 53.0, 0.7, 136.0, 3.0, 6.0, 905.0, 1988.0, 2631.0, 17.0, 3.0, 7.0, 1.0, 1.0, 0.0]
	 [563.0, 1.0, 0.5, 1.0, 2.0, 1.0, 41.0, 0.9, 145.0, 5.0, 6.0, 1263.0, 1716.0, 2603.0, 11.0, 2.0, 9.0, 1.0, 1.0, 0.0]


## Задание 2
###  Приведите test.csv к аналогичному виду, что и train

In [6]:
test = test\
    .filter(lambda row: row != test_first_row)\
    .map(lambda row: [float(el) for el in row.split(',')[1:]])
    
test_header = test_first_row.split(',')
beautiful_print(test, 3, test_header)

['id', 'battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g', 'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g', 'touch_screen', 'wifi']
	 [1043.0, 1.0, 1.8, 1.0, 14.0, 0.0, 5.0, 0.1, 193.0, 3.0, 16.0, 226.0, 1412.0, 3476.0, 12.0, 7.0, 2.0, 0.0, 1.0, 0.0]
	 [841.0, 1.0, 0.5, 1.0, 4.0, 1.0, 61.0, 0.8, 191.0, 5.0, 12.0, 746.0, 857.0, 3895.0, 6.0, 0.0, 7.0, 1.0, 0.0, 0.0]
	 [1807.0, 1.0, 2.8, 0.0, 1.0, 0.0, 27.0, 0.9, 186.0, 3.0, 4.0, 1270.0, 1366.0, 2396.0, 17.0, 10.0, 10.0, 0.0, 1.0, 1.0]


###Объединим train и test
###PS: нужно сделать средствами rdd pd.concat([train, test,], axis=0)

## Задание 3
###  Объедините данные

In [7]:
combined_rdd = train.union(test)

###Рассчитайте соотношение сторон экрана и его плотность пикселей

## Задание 4
###  Рассчитайте соотношение сторон экрана и найдите широкоформатные, выведите это распределение

In [8]:
# Расчет соотношения сторон экрана
aspect_ratio_rdd = combined_rdd.map(lambda x: (x[12]/x[11]) if x[11] != 0 else 0)


In [9]:
aspect_ratio_rdd.collect()[:5]

[37.8, 2.196685082872928, 1.3586698337292162, 1.46875, 1.0033112582781456]

In [10]:
# Вывод широкоформатных экранов
wide_screen_rdd = aspect_ratio_rdd.filter(lambda x: x > 1.78)

print(f"Широкоформатные экраны: {wide_screen_rdd.count()},\nНеширокоформатные экраны: {combined_rdd.count()-wide_screen_rdd.count()}")


Широкоформатные экраны: 1636,
Неширокоформатные экраны: 1364


## Задание 5
###  Рассчитайте плотность пикселей (лучше ppi), выведите это распределение

In [11]:
#import numpy as np

# Рассчитываем PPI
ppi_rdd = combined_rdd.map(lambda x: round(((x[11]**2 + x[12]**2)**0.5) / (((x[14]**2 + x[15]**2)**0.5)/2.54)))

# Выводим RDD с PPI
ppi_rdd.take(3)

[168, 321, 484]

In [12]:
# Вычисляем распределение PPI
distribution = ppi_rdd.countByValue()

# Выводим распределение PPI. Отсортируем и выведем топ-10
sorted_distribution = sorted(distribution.items(), key=lambda x: x[1], reverse=True)
sorted_distribution[:10]


[(209, 18),
 (225, 17),
 (292, 16),
 (197, 15),
 (258, 15),
 (138, 15),
 (199, 14),
 (285, 13),
 (122, 13),
 (176, 13)]

## Задание 6
###  Рассчитайте для train и test, затем объедините результат

In [13]:
ppi_train_rdd = train.map(lambda x: round(((x[11]**2 + x[12]**2)**0.5) / ((x[14] / 2.54) * (x[15] / 2.54)) if x[14] != 0 and x[15] != 0 else 0))
ppi_test_rdd = test.map(lambda x: round(((x[11]**2 + x[12]**2)**0.5) / ((x[14] / 2.54) * (x[15] / 2.54)) if x[14] != 0 and x[15] != 0 else 0)) 
ppi_all_rdd = ppi_train_rdd.union(ppi_test_rdd)